In [1]:
%pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX data into pandas dataframe
spacex_df = pd.read_csv("/content/spacex_launch_dash.csv")

# Get the min and max payload for the slider
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Create a Dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown to select Launch Site
    html.Div([
        dcc.Dropdown(id='site-dropdown',
                     options=[
                         {'label': 'All Sites', 'value': 'ALL'},
                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}
                     ],
                     value='ALL',
                     placeholder="Select a Launch Site here",
                     searchable=True)
    ]),

    html.Br(),

    # Pie chart to show the success count of launches
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Slider to select Payload range
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={i: str(i) for i in range(0, 10001, 1000)},
                    value=[min_payload, max_payload]),

    # Scatter chart to show correlation between Payload and Launch Success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback for the success pie chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # If ALL sites are selected, display a pie chart for all sites
        site_counts = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(site_counts, names='Launch Site', values='class', title='Launch Success Count for All Sites')
    else:
        # If a specific site is selected, display a pie chart for that site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_counts = filtered_df.groupby('Booster Version Category')['class'].sum().reset_index()
        fig = px.pie(site_counts, names='Booster Version Category', values='class', title=f"Launch Success Count for {entered_site}")
    return fig

# Callback for the scatter plot showing Payload vs Launch Success
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def update_scatter_plot(selected_site, payload_range):
    # Filter data based on selected payload range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]

    # If a specific site is selected, filter by site
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    # Create the scatter plot
    fig = px.scatter(
        filtered_df,
        x="Payload Mass (kg)",
        y="class",
        color="Booster Version Category",
        title=f"Payload Mass vs Launch Outcome for {selected_site if selected_site != 'ALL' else 'All Sites'}",
        labels={"Payload Mass (kg)": "Payload Mass (kg)", "class": "Launch Outcome"}
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>